In [69]:
from sympy import *

In [70]:
from icecream import ic
import pandas as pd

In [71]:
x = symbols('x')
y = symbols("y", cls=Function) # = Function('y')

In [72]:
points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
aFunction = 2 * y(x) + exp(x) - x
aInitialCondition = (0, 1/4)  # {y(0): 1 / 4}
bFunction = x - 1 + (x + 1)*y(x)
bInitialCondition = (0, 0)  # {y(0): 0}

In [73]:
def dsolveExactSolution(func, initialCondition=(0, 0), pointValues=()):
    equation = Eq(y(x).diff(x), func)
    initialCondition = {y(initialCondition[0]): initialCondition[1]}
    solution = dsolve(equation, y(x), ics=initialCondition)
    solution = solution.simplify()
    values = [round(solution.rhs.subs(x, p).evalf(10), 5) for p in pointValues]
    return solution, values

In [74]:
a1Solution, a1Values = dsolveExactSolution(aFunction, aInitialCondition, points)
a1Solution, a1Values

(Eq(y(x), x/2 + exp(2*x) - exp(x) + 1/4),
 [0.25000, 0.62042, 1.18372, 2.04800, 3.37749, 5.42077])

In [75]:
b1Solution, b1Values = dsolveExactSolution(bFunction, bInitialCondition, points)
b1Solution, b1Values

(Eq(y(x), (1 + sqrt(2)*sqrt(pi)*exp(1/2)*erf(sqrt(2)/2))*exp(x*(x + 2)/2) - sqrt(2)*sqrt(pi)*exp(x**2/2 + x + 1/2)*erf(sqrt(2)*(x + 1)/2) - 1),
 [0.0, -0.20283, -0.42446, -0.69114, -1.04407, -1.55268])

In [76]:
approximationPrecision = 10

In [77]:
def approximateAnalyticalSolution(func, initialCondition=(0, 0), Nn=5, pointValues=()):
    if Nn < 1:
        raise Exception("Wymagana minimum precyzja 1 pochodnej")
    x0 = initialCondition[0]
    y0 = initialCondition[1]
    # ic(func, x0, y0)
    # Wartosc y0
    yValues = [y0]
    # Pierwsza pochodna y(1)
    yPrim = func.subs(y(x), y0).subs(x, x0)
    yValues.append(yPrim)
    # Iteracyjnie obliczanie kolejnych pochodnych y(n)
    subsDict = {x: x0, y(x): y0}
    for n in range(2, Nn+1):
        func = diff(func, x)
        # Używanie poprzednich wartości pochodnych do podstawienia
        subsDict[diff(y(x), x, n-1)] = yValues[n-1]
        yn = func.subs(subsDict)
        yValues.append(yn)
    # Podstawienie Y(k)(x0) do wzoru
    taylorSeries = sum(yValues[k] / factorial(k) * ((x - x0) ** k) for k in range(Nn+1))
    # Podstawienie punktow do wzoru
    approximationSolutions = [round(taylorSeries.subs(x, p).evalf(10), 5) for p in pointValues]
    return taylorSeries, approximationSolutions


In [78]:
a2Solution, a2Values = approximateAnalyticalSolution(aFunction, aInitialCondition, approximationPrecision, points)
a2Solution, a2Values

(0.000281911375661376*x**10 + 0.00140817901234568*x**9 + 0.00632440476190476*x**8 + 0.0251984126984127*x**7 + 0.0875*x**6 + 0.258333333333333*x**5 + 0.625*x**4 + 1.16666666666667*x**3 + 1.5*x**2 + 1.5*x + 0.25,
 [0.25000, 0.62042, 1.18372, 2.04800, 3.37749, 5.42071])

In [79]:
b2Solution, b2Values = approximateAnalyticalSolution(bFunction, bInitialCondition, approximationPrecision, points)
b2Solution, b2Values

(-71*x**10/90720 - 43*x**9/18144 - 11*x**8/2016 - x**7/63 - x**6/36 - x**5/12 - x**4/12 - x**3/3 - x,
 [0, -0.20283, -0.42446, -0.69114, -1.04403, -1.55226])

In [80]:
def numericalSolution(func, initialCondition=(0, 0)):
    x0 = initialCondition[0]
    y0 = initialCondition[1]
    yValue = y0
    # akurat punkty roznia sie o 0.2 i jest 6 punktow
    h = 0.2
    pCount = 6

    yValues = [y0]
    for k in range(1, pCount):
        xK = x0 + (k-1) * h
        f = func.subs({x: xK, y(x): yValue})
        yValue = yValue + h * f
        yValues.append(yValue)
    return [round(yValue, 5) for yValue in yValues]

In [81]:
a3Values = numericalSolution(aFunction, aInitialCondition)
a3Values

[0.25, 0.55000, 0.97428, 1.58236, 2.45972, 3.72872]

In [82]:
b3Values = numericalSolution(bFunction, bInitialCondition)
b3Values

[0, -0.20000, -0.40800, -0.64224, -0.92776, -1.30175]

In [83]:
def createTable(tableName, columnHeaders, rowHeaders, tableValues):
    columns = [f"x{i} = {v}" for i, v in enumerate(columnHeaders)]
    table = pd.DataFrame(tableValues, columns=columns, index=rowHeaders)
    table.columns.name = tableName
    return table


In [84]:
aTable = createTable("(a)", points, ["RD", f"RA({approximationPrecision})", "RN"], [a1Values, a2Values, a3Values])
aTable

(a),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.25000,0.62042,1.18372,2.04800,3.37749,5.42077
RA(10),0.25000,0.62042,1.18372,2.04800,3.37749,5.42071
RN,0.25,0.55000,0.97428,1.58236,2.45972,3.72872


In [85]:
bTable = createTable("(b)", points, ["RD", f"RA({approximationPrecision})", "RN"], [b1Values, b2Values, b3Values])
bTable

(b),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.0,-0.20283,-0.42446,-0.69114,-1.04407,-1.55268
RA(10),0,-0.20283,-0.42446,-0.69114,-1.04403,-1.55226
RN,0,-0.20000,-0.40800,-0.64224,-0.92776,-1.30175
